In [1]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)
    
N_EPOCHS = 50
N_TRIALS = 30
BATCH_SIZE = 512

CONFIG = config_tests["proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [41]:
ds_1 = tfds.load("eurosat", split="train", as_supervised=True, shuffle_files=False)

x = []
y = []
for x_, y_ in ds_1.as_numpy_iterator():
    x.append(x_)
    y.append(y_)

2024-05-05 11:55:00.704545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [44]:
len(y)

27000

In [9]:
ds_2 = tfds.load("stl10", split="test", as_supervised=True, shuffle_files=False)
for x_, y_ in ds_2.as_numpy_iterator():
    x.append(x_)
    y.append(y_)

2024-05-05 11:11:58.167656: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [38]:
x = np.array(x, dtype=np.float32) / 255
y = np.array(y, dtype=np.float32)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]
train_size, len(x_val), len(x_test)

(18900, 4050, 4050)

In [39]:
9100 + 1950 + 1950

13000

In [6]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

In [7]:
model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["test_model_1"])
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
# model.save_weights("../models/default_model.weights.h5")
model.load_weights("../models/default_model.weights.h5")
model_weights = model.get_weights()

In [8]:
CONFIG

{'order_type': 'proba'}

In [14]:
model_scores = []
verbose = 0
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=0)

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)

    for i in range(N_EPOCHS):
        n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)

        samples_proba = utils.calculate_proba(
            model, x_train_sorted, y_train_sorted, counts, batch_size=BATCH_SIZE
        )

        samples_ids = utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])

        model.fit(
            x_train_sorted[samples_ids],
            y_train_sorted[samples_ids],
            # validation_data=(x_val, y_val),
            epochs=1,
            batch_size=BATCH_SIZE,
            verbose=verbose,
        )

    model.fit(
        x_train, y_train, validation_data=(x_val, y_val), epochs=500, batch_size=BATCH_SIZE, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=10)], 
        verbose=verbose
    )
            
    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
    model_scores.append(accuracy)
    print("Mean:", np.mean(model_scores), " Median: ", np.median(model_scores))

  0%|          | 0/30 [00:00<?, ?it/s]

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8317 - loss: 0.5113


  3%|▎         | 1/30 [02:04<1:00:14, 124.63s/it]

Mean: 0.8392592668533325  Median:  0.8392592668533325
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8100 - loss: 0.6265


  7%|▋         | 2/30 [04:12<59:05, 126.62s/it]  

...

Mean: 0.8126014173030853  Median:  0.8161728382110596
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8387 - loss: 0.5375


 97%|█████████▋| 29/30 [1:03:54<02:14, 134.70s/it]

Mean: 0.8135632235428383  Median:  0.8204938173294067
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8420 - loss: 0.5117


100%|██████████| 30/30 [1:06:18<00:00, 132.61s/it]

Mean: 0.8145761370658875  Median:  0.8223456740379333


In [15]:
np.min(model_scores)

0.7212345600128174

In [11]:
print("Mean:", np.mean(model_scores), " Median: ", np.median(model_scores))

Mean: 0.6160512844721476  Median:  0.6143589615821838


In [18]:
print("Mean:", np.mean(model_scores), " Median: ", np.median(model_scores))

Mean: 0.5817307829856873  Median:  0.5800000131130219


In [17]:
CONFIG

{'order_type': 'proba'}

In [16]:
model_scores

[0.8392592668533325,
 0.8106172680854797,
 0.8518518805503845,
 0.8118518590927124,
 0.8627160787582397,
 0.7886419892311096,
 0.8288888931274414,
 0.8254321217536926,
 0.82419753074646,
 0.8402469158172607,
 0.7856789827346802,
 0.8083950877189636,
 0.8318518400192261,
 0.8059259057044983,
 0.8204938173294067,
 0.8283950686454773,
 0.7212345600128174,
 0.760493814945221,
 0.8012345433235168,
 0.7787654399871826,
 0.8548148274421692,
 0.8037037253379822,
 0.8424691557884216,
 0.7701234817504883,
 0.8392592668533325,
 0.7866666913032532,
 0.7990123629570007,
 0.8306173086166382,
 0.8404937982559204,
 0.843950629234314]

In [17]:
CONFIG

{'order_type': 'fixed'}

In [26]:
model_scores_random = []
verbose = 0

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)

    model.fit(
        x_train, y_train, validation_data=(x_val, y_val), epochs=500, batch_size=BATCH_SIZE, verbose=verbose, shuffle=True, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=35)]
    )

    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
    model_scores_random.append(accuracy)
    print("Mean:", np.mean(model_scores_random), " Median: ", np.median(model_scores_random))

  0%|          | 0/30 [00:00<?, ?it/s]

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8059 - loss: 0.5594


  3%|▎         | 1/30 [00:48<23:24, 48.45s/it]

Mean: 0.8125925660133362  Median:  0.8125925660133362
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8031 - loss: 0.6061


  7%|▋         | 2/30 [01:35<22:12, 47.58s/it]

...

Mean: 0.792786602463041  Median:  0.8011111319065094
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7810 - loss: 0.7174


 97%|█████████▋| 29/30 [24:12<00:49, 49.21s/it]

Mean: 0.7925159705096277  Median:  0.7980247139930725
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7862 - loss: 0.6834


100%|██████████| 30/30 [25:06<00:00, 50.23s/it]

Mean: 0.7924938340981801  Median:  0.7977778017520905


In [27]:
model_scores_random

[0.8125925660133362,
 0.8041975498199463,
 0.8212345838546753,
 0.826913595199585,
 0.7743209600448608,
 0.7866666913032532,
 0.7980247139930725,
 0.7106173038482666,
 0.8066666722297668,
 0.7432098984718323,
 0.8530864119529724,
 0.7911111116409302,
 0.8212345838546753,
 0.7693827152252197,
 0.7130864262580872,
 0.7683950662612915,
 0.8397530913352966,
 0.7975308895111084,
 0.8274074196815491,
 0.7614814639091492,
 0.8493826985359192,
 0.7209876775741577,
 0.7834568023681641,
 0.8081481456756592,
 0.8451851606369019,
 0.8212345838546753,
 0.8358024954795837,
 0.7069135904312134,
 0.7849382758140564,
 0.7918518781661987]

In [13]:
np.mean(model_scores), np.mean(model_scores_random)

NameError: name 'model_scores' is not defined

In [16]:
import scipy.stats
scipy.stats.ttest_ind(model_scores, model_scores_random)

TtestResult(statistic=4.752362071023242, pvalue=1.3666137160185319e-05, df=58.0)